In [33]:
# Constants
DATA_PATH = './data/'
DATASET_PATH = DATA_PATH + 'smartphone+based+recognition+of+human+activities+and+postural+transitions/'

In [34]:
# For loading the labels file
class SectionData():
    def __init__(self, str):
        new_data= str.split(' ')
        self.exp_num = new_data[0]
        self.user_num = new_data[1]
        self.activity = new_data[2]
        self.start_point = new_data[3]
        self.end_point = new_data[4]
    
    def __str__(self):
        return str("(" + self.exp_num + ", " + self.user_num + ", " + self.activity + ", " + self.start_point + ", " + self.end_point.strip("\n") + ")")

class LabelsFile():
    def __init__(self):
        self.data = open(DATASET_PATH + "RawData/labels.txt", 'r').readlines()
        self.i = 0
    
    def __iter__(self):
        self.i = 0
        return self
    
    def __next__(self):
        if self.i < len(self.data):
            current = self.data[self.i]
            self.i += 1
            return SectionData(current)
        raise StopIteration 

In [35]:
# For handling raw data

def load(data):
    pass

In [36]:
# Main function
def main():
    # 1. Grab the labels.txt file telling you what each data section means (x)
    # 2. For each section: (x)
        # i. Load that section's respective data
        # ii. For each window in that section:
            # a. Apply noise filters on the window
            # b. Compute features from the window
            # c. Store those feature values in an array
        # iii. Store all the features and their respective data in a file
    labels_file = LabelsFile()
    for section in labels_file:
        raw_data = load(section)
        pass
    pass

main()